用兩種方式達到對明亮度做直方圖均衡處理\
水平鏡像 + 縮放處理 (0.5 倍)\
畫出人物矩形邊框

In [2]:
import cv2
import numpy as np
img = cv2.imread('lena.png')
top_left_corner=(60,40)
bottom_right_corner=(420,510)

# 矩形

In [7]:
img_rect = img.copy()
cv2.rectangle(
    img_rect, #picture
    top_left_corner, #rectangle place
    bottom_right_corner,
    (255,0,0), #line color
    3         #line thinkness
)
while True:
    cv2.imshow('image', img_rect)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

# 線

In [10]:
img_line=img.copy()
cv2.line(
    img_line,
    top_left_corner,
    bottom_right_corner,
    (255,0,0),
    3
)
while True:
    cv2.imshow('image', img_line)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

# 文字

In [19]:
img_text=img.copy()
cv2.putText(
    img_text,
    "today is Monday.",
    (60,40), #⽂字左下⾓位置
    7, #字型
    1.5, #字體⼤⼩
    (255,255,255), #字體顏⾊
    2    #字體粗細
)
while True:
    cv2.imshow('image', img_text)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

# 解法一

In [47]:
img_hw = img.copy()
point1 = (60, 40)
point2 = (420, 510)

"""
對明亮度做直方圖均衡
"""
# 原始 BGR 圖片轉 HSV 圖片
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)

# 對明亮度做直方圖均衡 -> 對 HSV 的 V 做直方圖均衡
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])

# 將圖片轉回 BGR
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

"""
畫出人物矩形邊框
"""
cv2.rectangle(img_hw, point1, point2, (0, 0, 255), 3)

"""
水平鏡像 + 縮放處理 (0.5 倍)
"""
# 水平鏡像 (圖片)
img_hw = img_hw[:, ::-1, :]

# 縮放處理
img_hw = cv2.resize(img_hw, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

while True:
    cv2.imshow('image', img_hw)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

# 解法二

In [46]:
img_hw = img.copy()
point1=[60,40]
point2=[420,510]

"""
對明亮度做直方圖均衡
"""
# 原始 BGR 圖片轉 HSV 圖片
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
# 對明亮度做直方圖均衡 -> 對 HSV 的 V 做直方圖均衡
img_hw[...,-1]=cv2.equalizeHist(img_hw[:,:,-1])
# 將圖片轉回 BGR
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

"""
水平鏡像
"""
h, w = img_hw.shape[:2]
# 圖片鏡像
img_hw = img_hw[:,::-1,:]
# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
point1[0]=-60
point2[0]=-420

"""
縮放處理 (0.5 倍)
"""
fx=0.5
fy=0.5
resize_col=int(img_hw.shape[1]*fx)
resize_row=int(img_hw.shape[0]*fy)
M_scale=np.array([[0.5,0,0],[0,0.5,0]])
img_hw = cv2.warpAffine(img_hw,M_scale,(resize_col,resize_row)) #需要图像、变换矩阵、变换后的大小
# 把左上跟右下轉為矩陣型式
bbox = np.array((point1, point2), dtype=np.float32)
print('M_scale.shape={}, bbox.shape={}'.format(M_scale.shape, bbox.shape))
# 做矩陣乘法可以使用 `np.dot`, 為了做矩陣乘法, M_scale 需要做轉置之後才能相乘
homo_coor_result = np.dot(M_scale.T, bbox)
homo_coor_result = homo_coor_result.astype('uint8') #等比例縮小外加的框框
print(homo_coor_result)
#homo_coor_result轉成cv2.rectangle要得tuple
scale_point1 = tuple(homo_coor_result[0])
scale_point2 = tuple(homo_coor_result[1])
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))
# 畫圖
cv2.rectangle(img_hw, scale_point1, scale_point2, (0, 0, 255), 3)

while True:
    cv2.imshow('image', img_hw)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

M_scale.shape=(2, 3), bbox.shape=(2, 2)
[[226  20]
 [ 46 255]
 [  0   0]]
origin point1=[-60, 40], origin point2=[-420, 510]
resize point1=(226, 20), resize point2=(46, 255)


# 解法三
用解法1下去改\
把矩型邊框用遮罩的方式呈現，使用同樣處理圖片的方式處理遮罩 最後再找遮罩的左上跟右下的點的位置\
這邊會用到許多沒提過的東西，所以當作 optional

(512,)

參考解

In [5]:
img_hw = img.copy()
point1 = [60, 40]
point2 = [420, 510]

# 2D mask
img_mask = np.zeros_like(img_hw)
img_mask[40:510, 60:420, ...] = 255

"""
對明亮度做直方圖均衡
"""
# 原始 BGR 圖片轉 HSV 圖片
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)

# 對明亮度做直方圖均衡 -> 對 HSV 的 V 做直方圖均衡
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])

# 將圖片轉回 BGR
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

"""
水平鏡像
"""
h, w = img_hw.shape[:2]

# 圖片鏡像
img_hw = img_hw[:, ::-1, :]
img_mask = img_mask[:, ::-1, :]

"""
縮放處理 (0.5 倍)
"""
fx = 0.5
fy = 0.5
img_hw = cv2.resize(img_hw, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)
img_mask = cv2.resize(img_mask, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)

# 取得遮照的左上跟右下的點
all_h_coor, all_w_coor, all_c_coor = np.where(img_mask)
scale_point1 = (min(all_w_coor), min(all_h_coor))
scale_point2 = (max(all_w_coor), max(all_h_coor))
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))

# 畫圖
cv2.rectangle(img_hw, scale_point1, scale_point2, (0, 0, 255), 3)

while True:
    cv2.imshow('image', img_hw)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

origin point1=[60, 40], origin point2=[420, 510]
resize point1=(46, 20), resize point2=(225, 254)
